# Тестовое задание для *Tectum*
### Рогачев Максим, 09.02.24

**Задача:** реализовать автоматическое тестирование стратегии, описанной тут - https://www.youtube.com/watch?v=aGKjGtK5kJw&ab_channel=%D0%A7%D0%B5%D1%81%D1%82%D0%BD%D1%8B%D0%B9%D0%A2%D1%80%D0%B5%D0%B9%D0%B4%D0%B5%D1%80. Собрать необходимую статистику по проделанным сделкам на разных tf. 

Для выполнения задачи я решил использовать библиотеку backtrader - новый для себя инструмент, до этого я проводил тестирование вручную. Backtrader удобен тем, что предоставляет много готовых индикаторов, также упрощает процесс анализа. 

In [186]:
import pandas as pd
import backtrader as bt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Итак, первым делом необходимо подготовить данные. Данные скачиваются с бинанс не единым файлом, а отдельным файлом за каждый месяц, поэтому данные нужно объединить. Также необходимо обработать даты, которые изначально хранятся в формате timestamp. 

In [187]:
def make_dataset(file_name):
    '''
    Функция считывает все файлы для конкретного tf и объединяет их в единый датасет
    file_name: общее название файлов для конкретного tf, str
    '''
    data = pd.DataFrame()
    for i in range(1, 13): #проходим данные за каждый месяц
        tmp = pd.read_csv(file_name + f"{i}.csv", usecols=range(5), names=['open_time', 'open', 'high', 'low', 'close'])
        tmp['open_time'] = pd.to_datetime(tmp['open_time'], unit='ms') #преобразуем столбец с датой
        data = pd.concat([data, tmp]) #объединяем датасеты
    data = data.reset_index(drop=True)  
    return data

datasets = []
for file_name in ['APEUSDT-5m-2023-', 'APEUSDT-15m-2023-', 'APEUSDT-1h-2023-']: #для каждого tf вызываем функцию make_dataset
    datasets.append(make_dataset(file_name))
data_5_mins, data_15_mins, data_1_hour = datasets[0], datasets[1], datasets[2] #разделяем датасеты по признаку tf

data_5_mins = data_5_mins.drop(23767) #в 5 минутном датасете есть одна строчка, 
                                      #от коротой backtrader падает, 
                                      #потому что в ней совпадают значения всех столбцов

Далее можно перейти к реализации класса стратегии и торговой логики. 

**Торговый алгоритм:**
1. Получив очередную цену закрытия, проверить предыдущую цену на пиковость(лок. максимум или минимум)
2. Если позиция на данный момент не открыта, то проверяем условия для открытия сделки. Если они выполнены, то открываем сделку и рассчитываем стоп-лосс и тейк-профит. 
3. Stop-loss ставится ниже или выше пика на значение stop_loss_offset. При этом, если stop-loss выходит за границы stop_loss_min или stop_loss_max, то он пересчитывается от предыдущего пика. Процесс пересчёта продолжается до тех пор, пока stop-loss не попадёт четко в интервал. Если же такого пика не обнаружится, то stop-loss будет установлен в занчение stop_loss_min или в stop_loss_max, в зависимости от того, за границы какого ограничения он вышел. 
4. Take-profit всегда рассчитывается как stop-loss умноженный на значение risk_reward_ratio 
5. Если на данный момент есть открытая позиция, то происходит проверка, не пора ли её закрыть по stop-loss или take-profit. Если есть такая необходимость, то сделка закрывается. 


In [188]:
class StochRSIMACD(bt.Strategy):
    
    params = (
        ('stoch_period', 14), #период стохастика
        ('rsi_period', 14), #период rsi
        ('macd1', 12), #период первой скользящей средней MACD
        ('macd2', 26), #период второй скользящей средней MACD
        ('macdsig', 9), #период сигнальной линии MACD 
        ('stop_loss_offset', 0.01), #отступ от пиков(локальных минимумов и максимумов), 0.01 = 1%
        ('risk_reward_ratio', 2.25), #соотношение прибыли к риску
        ('min_stop_loss', 1), #минимальное значение стоп-лосса в процентах
        ('max_stop_loss', 4), #максимальное значение stop_loss в процентах
        ('time_frame', 5) #time frame в минутах
    )

    def __init__(self):
        #инициализация индикаторов
        self.stoch = bt.indicators.Stochastic(period=self.params.stoch_period, period_dfast=3)
        self.rsi = bt.indicators.RSI(period=self.params.rsi_period)
        self.macd = bt.indicators.MACD(period_me1=self.params.macd1,
                                       period_me2=self.params.macd2,
                                       period_signal=self.params.macdsig)
        self.dataclose = self.datas[0].close #для удобства обращения к последней цене закрытия
        self.local_minimums = [] #хранение локальных минимумов
        self.local_maximums = [] #хранение локальных максимумов
        #хранение инфы об ордерах
        self.order_info = {'risk_reward_ratio': [], #соотношение прибыли к риску
                           'stop_loss, %': [], #стоп-лосс в процентах
                           'take_profit, %': [], #тейк-профит в процентах
                           'result': [], #PnL ордера
                           'duration, bars': [] #длительность ордера в барах(число свеч)
                           }
    
    def close_trade(self, close_price):
        '''
        Функция закрытия ордеров, требуется для сохранения информации
        close_price: цена закрытия. Либо stop_loss, либо take_profit
        '''
        self.order_info['result'].append((close_price - self.open_price) * self.order.executed.size) #сохраняем PnL
        self.order_info['duration, bars'].append((self.datas[0].datetime.datetime(0).timestamp() - self.open_time) / self.params.time_frame / 60) #сохраняем длительность
        self.order = self.close() #закрываем сделку

    def write_order_info(self):    
        '''
        Функция записи информации об ордере при его открытии
        '''
        self.order_info['risk_reward_ratio'].append(self.params.risk_reward_ratio) #запись соотношения прибыли к риску
        self.order_info['stop_loss, %'].append(self.stop_loss_perc * 100) #запись стоп-лосса в процентах
        self.order_info['take_profit, %'].append(self.take_profit_perc) #запись тейк-профита в процентах
        self.open_time = self.datas[0].datetime.datetime(0).timestamp() #сохранение времени открытия

    def calculate_perc(self, open_price, close_price):
        '''
        Функция вычисления процентов
        '''
        return (open_price - close_price) / open_price   

    def next(self):
        '''
        Основная функция итерации по данным. Работает для каждой строки в датасете
        ''' 
        #вычисляем локальные минимумы и максимумы
        if self.dataclose[-2] > self.dataclose[-1] and self.dataclose[0] > self.dataclose[-1]:
            self.local_minimums.append(self.dataclose[-1])
        elif self.dataclose[-2] < self.dataclose[-1] and self.dataclose[0] < self.dataclose[-1]:
            self.local_maximums.append(self.dataclose[-1])
        
        if not self.position: #если нет открытых ордеров
            #проверяем условия для открытия сделки на покупку
            if len(self.local_minimums) >= 1 and self.rsi[0] > 50 and self.stoch[0] > 20 and self.macd.macd[0] >= self.macd.signal[0]:
                self.order = self.buy() #покупаем
                self.open_price = self.dataclose[0] #цена открытия
                self.stop_loss = self.local_minimums[-1] * (1 - self.params.stop_loss_offset) #расчет стоп-лосса
                
                #проверяем, что стоп-лосс больше минимального процента
                i = -2
                while i*(-1) <= len(self.local_minimums) and \
                (self.calculate_perc(self.open_price, self.stop_loss) * 100 < self.params.min_stop_loss or \
                 self.calculate_perc(self.open_price, self.stop_loss) * 100 > self.params.max_stop_loss):
                    self.stop_loss = self.local_minimums[i] * (1 - self.params.stop_loss_offset) #при необходимости отсчитываем стоп от предыдущего пика
                    i -= 1
                #если на любом пике мы выходим за границы stop-loss        
                if self.calculate_perc(self.open_price, self.stop_loss) * 100 < self.params.min_stop_loss:
                    self.stop_loss = self.open_price * (1 - self.params.min_stop_loss / 100)    
                elif self.calculate_perc(self.open_price, self.stop_loss) * 100 > self.params.max_stop_loss:
                    self.stop_loss = self.open_price * (1 - self.params.max_stop_loss / 100)        
                
                self.stop_loss_perc = self.calculate_perc(self.open_price, self.stop_loss) #расчет стопа в процентах
                self.take_profit = self.open_price * (1 + self.stop_loss_perc * self.params.risk_reward_ratio) #расчёт тейк-профита в зависимости от стопа
                self.take_profit_perc = self.stop_loss_perc * self.params.risk_reward_ratio * 100 #расчет тейк-профита в процентах
                self.write_order_info() #вызов функции записи инфы
            
            #проверяем условия для открытия сделки на продажу
            elif len(self.local_maximums) >= 1 and self.rsi[0] < 50 and self.stoch[0] < 80 and self.macd.macd[0] <= self.macd.signal[0]:
                self.order = self.sell() #продаём
                self.open_price = self.dataclose[0] #цена открытия
                self.stop_loss = self.local_maximums[-1] * (1 + self.params.stop_loss_offset) #расчет стоп-лосса
                
                #проверяем, что стоп-лосс больше минимального процента
                i = -2
                while i*(-1) <= len(self.local_maximums) and \
                (self.calculate_perc(self.stop_loss, self.open_price) * 100 < self.params.min_stop_loss or \
                 self.calculate_perc(self.stop_loss, self.open_price) * 100 > self.params.max_stop_loss):
                    self.stop_loss = self.local_maximums[i] * (1 + self.params.stop_loss_offset) #при необходимости отсчитываем стоп от предыдущего пика
                    i -= 1
                #если на любом пике мы выходим за границы stop-loss    
                if self.calculate_perc(self.stop_loss, self.open_price) * 100 < self.params.min_stop_loss: 
                    self.stop_loss = self.open_price * (1 + self.params.min_stop_loss / 100) 
                elif self.calculate_perc(self.stop_loss, self.open_price) * 100 > self.params.max_stop_loss:
                    self.stop_loss = self.open_price * (1 + self.params.max_stop_loss / 100)     
                
                self.stop_loss_perc = self.calculate_perc(self.stop_loss, self.open_price) #расчет стопа в процентах
                self.take_profit = self.open_price * (1 - self.stop_loss_perc * self.params.risk_reward_ratio) #расчёт тейк-профита в зависимости от стопа
                self.take_profit_perc = self.stop_loss_perc * self.params.risk_reward_ratio * 100 #расчет тейк-профита в процентах
                self.write_order_info() #вызов функции записи инфы
        
        else: #если сделка открыта
            # проверяем, не нужно ли закрыть её по стопу
            if (self.dataclose[0] <= self.stop_loss and self.order.isbuy()) or (self.dataclose[0] >= self.stop_loss and self.order.issell()):
                self.close_trade(self.stop_loss)   
            # проверяем, не нужно ли закрыть её по тейк-профиту    
            elif (self.dataclose[0] >= self.take_profit and self.order.isbuy()) or (self.dataclose[0] <= self.take_profit and self.order.issell()):
                self.close_trade(self.take_profit)               

Следующая функция подтягивает наши данные, инициализирует класс и стратегии и запускает тест.

In [189]:
def start_backtesting(data, stop_loss_offset, risk_reward_ratio, min_stop_loss, max_stop_loss, tf):
    '''
    Функция запуска backtesting. Инициализирует стратегию и данные, 
    запускает отработку и возвращает датасет с информацией об ордерах
    data: датасет с данными, pd.DataFrame
    stop_loss_offset: отступ от пика при установке stop_loss, 0.01=1%
    risk_reward_ratio: соотношение прибыли к риску, float
    min_stop_loss: минимальный stop-loss в процентах
    tf: timeframe данных в минутах
    return: датасет с информацией о сделках, pd.DataFrame

    Столбцы в возвращаемом дата фрейме:  
    risk_reward_ratio - соотношение прибыли к риску
    stop_loss - стоп-лосс в процентах
    take_profit - тейк-профит в процентах
    result - PnL ордера
    duration, bars - длительность ордера в барах(число свеч)    
    '''
    cerebro = bt.Cerebro()
    cerebro.addstrategy(StochRSIMACD, #инициализируем стратегию
                        stop_loss_offset=stop_loss_offset, 
                        risk_reward_ratio=risk_reward_ratio, 
                        min_stop_loss=min_stop_loss,
                        max_stop_loss=max_stop_loss,
                        time_frame=tf)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=20) #каждую сделку открываем на 20% от банка

    data_parsed = bt.feeds.PandasData(dataname=data, #приводим данные в необходимый формат
                                      datetime=0, 
                                      open=1, 
                                      high=2, 
                                      low=3, 
                                      close=4, 
                                      volume=None, 
                                      openinterest=None)
    cerebro.adddata(data_parsed) #добавляем данные

    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer') #добавляем аналитику

    results = cerebro.run() #запускаем тестирование

    #trade_analysis = results[0].analyzers.tradeanalyzer.get_analysis() #получаем анализ по торговле
    
    results[0].order_info['result'].append('*') #ставим заглушки для незакрытых сделок
    results[0].order_info['duration, bars'].append('*') #ставим заглушки для незакрытых сделок
    trade_data = pd.DataFrame(results[0].order_info) #записываем информацию об ордерах

    return trade_data

Ещё пара вспомогательных функций:

In [190]:
def find_max_sub(df, is_positive):   
    '''
    Вспомогательная функция для расчёта максимального количества подряд идущих
    прибыльных и убыточных сделок
    df: датафрейм для расчёта, pd.DataFrame
    is_positive: если True, то считаются прибыльные сделки. В противном случае - убыточные
    return: максимальное кол - во подряд идущих сделок
    '''
    if is_positive: 
        df['tmp'] = np.where(df['result'] > 0, 1, 0)
    else:
        df['tmp'] = np.where(df['result'] < 0, 1, 0)  
    df['block'] = (df['tmp'].shift(1) != df['tmp']).astype(int).cumsum()
    max_in_a_row = df[df['tmp'] == 1].groupby('block').count().max().values[0]
    return max_in_a_row

In [195]:
def write_strategy_info(data, tf):
    '''
    Функция для вывода статистики о сделках
    '''
    print('Длительность проведения бектеста:', int(data[:250]['duration, bars'].sum() * tf / 60 / 24), 'дня')
    print('Количество закрытых сделок:', len(data[:250]))
    print('Общий PnL:', round(data[:250]['result'].sum(), 2))
    print('Максимальное число прибыльных сделок в ряд:', find_max_sub(data[:250], is_positive=True))
    print('Максимальное число убыточных сделок в ряд:', find_max_sub(data[:250], is_positive=False))
    print('WinRate: ', round((data[:250]['result'] > 0).sum() / len(data[:250]) * 100, 2), '%', sep='')

Стоит уточнить, что по умолчанию наш начальный баланс - 10000 у.е., а по выставленным настройкам каждая сделка открывается на 20% от банка. 

Следующие ячейки запускают тестирование для каждого из 3 таймфреймов. Обратите внимание, что при вызове функции можно менять все параметры, кроме data и tf. 

trade_df_{time_frame} - датафрейм, содержащий всю информацию о сделках для конкретного таймфрейма.

Правильно выставленные параметры могут обеспечить достаточно высокий Pnl на любом таймфрейме, однако от этого падает количество сделок. В этом задании я взял самые дефолтные параметры, описанные в видео, прогнал годовые данные для каждого таймфрейма и взял первые 250 сделок. 

In [192]:
trade_df_5_mins = start_backtesting(data=data_5_mins, stop_loss_offset=0.005, risk_reward_ratio=1.5, min_stop_loss=1, max_stop_loss=10, tf=5)
write_strategy_info(trade_df_5_mins, tf=5)

Длительность проведения бектеста: 133 дня
Количество закрытых сделок: 250
Общий PnL: 32.85
Максимальное число прибыльных сделок в ряд: 6
Максимальное число убыточных сделок в ряд: 10
WinRate: 41.2%


In [193]:
trade_df_15_mins = start_backtesting(data=data_15_mins, stop_loss_offset=0.005, risk_reward_ratio=1.5, min_stop_loss=1, max_stop_loss=10, tf=15)
write_strategy_info(trade_df_15_mins, tf=5)

Длительность проведения бектеста: 95 дня
Количество закрытых сделок: 250
Общий PnL: -1481.09
Максимальное число прибыльных сделок в ряд: 7
Максимальное число убыточных сделок в ряд: 13
WinRate: 36.0%


In [194]:
trade_df_1_hour = start_backtesting(data=data_1_hour, stop_loss_offset=0.005, risk_reward_ratio=1, min_stop_loss=0.5, max_stop_loss=4, tf=60)[:-1]
write_strategy_info(trade_df_1_hour, tf=60)

Длительность проведения бектеста: 169 дня
Количество закрытых сделок: 250
Общий PnL: -1813.78
Максимальное число прибыльных сделок в ряд: 6
Максимальное число убыточных сделок в ряд: 7
WinRate: 42.8%


Подводя итог, могу предложить свои варианты по улучшению перфоманса стратегии:

1. Как я уже писал, правильно выставленные параметры сильно повышают PnL, однако уменьшают кол-во сделок, так как обычно правильные параметры - высокое соотношение прибыли и риска и высокий минимальный стоп-лосс. Из - за этого сделки долго висят открытыми, следовательно меньше сделок открывается. Отсюда рождается идея ограничения продолжительности сделок.

2. Возможно, есть смысл ввести ограничение сверху для тейк-профита. 